In [1]:
import numpy as np
import pandas as pd
import re
import json

import os

In [2]:
# enter directory path
path = '/data/dec2020'

In [9]:
data_folder = os.getcwd()+path
list_of_relevant_files = []

for f in sorted(os.listdir(data_folder)):
    # filter out system files
    if f != '.DS_Store':
        list_of_relevant_files.append(f)

In [10]:
len(list_of_relevant_files)

3

In [11]:
list_of_relevant_files

['zameen_12-01-2020.json', 'zameen_12-15-2020.json', 'zameen_12-30-2020.json']

In [12]:
# read files as json
list_of_dicts = []

for f in list_of_relevant_files:
    print(f)
    ff = open(data_folder+'/'+f)
    ff = ff.read()
    
    ff = '{"properties": ['+ff+']}'
    
    # add comma between brackets
    regex = re.compile("\}\{")
    ff = re.sub(regex, "}, {", ff)
    
    # for appended files
#     regex_append = re.compile("\}\\n\{")
#     ff = re.sub(regex_append, "}, {", ff)

    ff_json = json.loads(ff)
    list_of_dicts.append(ff_json)

zameen_12-01-2020.json
zameen_12-15-2020.json
zameen_12-30-2020.json


In [13]:
list_of_dicts[:1][0]['properties'][0]['Details']

['Type',
 'Flat',
 'Price',
 'PKR',
 '52 Lakh',
 'Location',
 'Bahria Town Karachi, Karachi, Sindh',
 'Bath(s)',
 '2',
 'Area',
 '4.9 Marla',
 'Purpose',
 'For Sale',
 'Bedroom(s)',
 '2',
 'Added',
 '28 minutes ago']

### Clean Description

In [14]:
for d in list_of_dicts:
    for n in range(len(d['properties'])):
        d['properties'][n]['Description'] = d['properties'][n]['Description'].replace('•', '').replace('>', '').replace('\t', '')
        d['properties'][n]['Description'] = re.sub("\s{2}", " ", d['properties'][n]['Description'])
        d['properties'][n]['Description'] = re.sub("\s{3,10}", "", d['properties'][n]['Description'])

### Convert Details from list into dict

In [15]:
for d in list_of_dicts:
    for n in range(len(d['properties'])):
        z = d['properties'][n]['Details']
        if type(z) == list:
            # plots don't have PKR value, so need to move indices by one
            if z[3] == 'Contact for price':
                details = {
                    z[10]: z[11],
                    z[0]: z[1],
                    z[2]: z[3], 
                    z[4]: z[5],
                    z[8]: z[9],
                    z[12]: z[13],
                    z[6]: z[7],
                    z[14]: z[15]
                }
            else:
                details = {
                    z[11]: z[12],
                    z[0]: z[1], 
                    z[2]: z[4],
                    z[5]: z[6],
                    z[9]: z[10],
                    z[13]: z[14],
                    z[7]: z[8],
                    z[15]: z[16]
                }
            d['properties'][n]['Details'] = details

### Amenities -> Year of Construction

In [17]:
# test
# re.findall('Built in year : ([0-9]{4})', ff_json['properties'][1]['Amenities'])[0]

In [18]:
for d in list_of_dicts:
    for n in range(len(d['properties'])):
        year_of_construction = re.findall('Built in year : ([0-9]{4})', d['properties'][n]['Amenities'])

        if year_of_construction:
            year_of_construction = year_of_construction[0]
        else:
            year_of_construction = np.nan

        d['properties'][n]['Year of Construction'] = year_of_construction

In [19]:
for d in list_of_dicts:
    for n in range(len(d['properties'])):
        servant_quarters = re.findall('Servant Quarters : ([0-9]{1})', d['properties'][n]['Amenities'])

        if servant_quarters:
            servant_quarters = servant_quarters[0]
        else:
            servant_quarters = np.nan

        d['properties'][n]['Servant Quarters'] = servant_quarters

In [20]:
for d in list_of_dicts:
    for n in range(len(d['properties'])):
        parking_spaces = re.findall('Parking Spaces : ([0-9]{1})', d['properties'][n]['Amenities'])

        if parking_spaces:
            parking_spaces = parking_spaces[0]
        else:
            parking_spaces = np.nan

        d['properties'][n]['Parking Spaces'] = parking_spaces

In [21]:
for d in list_of_dicts:
    for n in range(len(d['properties'])):
        double_glazed_windows = re.findall('Double Glazed Windows', d['properties'][n]['Amenities'])

        if double_glazed_windows:
            double_glazed_windows = "Yes"
        else:
            double_glazed_windows = "No"

        d['properties'][n]['Double Glazed Windows'] = double_glazed_windows

In [22]:
for d in list_of_dicts:
    for n in range(len(d['properties'])):
        central_ac = re.findall('Central Air Conditioning', d['properties'][n]['Amenities'])

        if central_ac:
            central_ac = "Yes"
        else:
            central_ac = "No"

        d['properties'][n]['Central AC'] = central_ac

In [23]:
for d in list_of_dicts:
    for n in range(len(d['properties'])):
        park = re.findall(r'\bnear park\b', d['properties'][n]['Description'])

        if park:
            park_present = "Yes"
        else:
            park_present = "No"

        d['properties'][n]['Park'] = park_present

In [24]:
for d in list_of_dicts:
    for n in range(len(d['properties'])):
        jacuzzi = re.findall('jacuzzi', d['properties'][n]['Description'])

        if jacuzzi:
            jacuzzi_present = "Yes"
        else:
            jacuzzi_present = "No"

        d['properties'][n]['Jacuzzi'] = jacuzzi_present

In [28]:
# find how many listings have specific details
# results = []

# for n in range(len(ff_json['properties'])):
#     result = re.findall('Central Air Conditioning', ff_json['properties'][n]['Amenities'])
#     if result:
#         results.append(n)

In [29]:
# len(results)

In [32]:
list_of_dicts[0]['properties'][0]

{'Property ID': 'Flat 26526717',
 'Title': 'Beautiful Apartment Is Available For Sale In Reasonable Price',
 'Subtitle': 'Bahria Heights, Bahria Town Karachi, Karachi, Sindh',
 'Details': {'Purpose': 'For Sale',
  'Type': 'Flat',
  'Price': '52 Lakh',
  'Location': 'Bahria Town Karachi, Karachi, Sindh',
  'Area': '4.9 Marla',
  'Bedroom(s)': '2',
  'Bath(s)': '2',
  'Added': '28 minutes ago'},
 'Description': 'Salaam Estate Offers:- Attractive Apartment Available for Sale at Bahria Heights, Bahria Town Features:- It consists of 2 Bed Rooms with attached Bath - TV Lounge - Dinning - Kitchen - Store Room - Open terrace - Washing Area - Reserved Car Parking Facilities / Attractions - High Speed Elevator - Reception Area - Fully Secured - Ready to Move - Near to Main Entrance - Near to Theme Park & Danzoo Contact us for more Details.',
 'Amenities': 'Bedrooms : 2 Bathrooms : 2 Kitchens : 1 Lounge or Sitting Room Broadband Internet Access Satellite or Cable TV Ready',
 'Agency': 'Salaam Est

In [33]:
len(list_of_dicts)

3

### Convert to Dataframe

In [34]:
dff = pd.DataFrame.from_dict(list_of_dicts[0]['properties'])

In [35]:
for d in list_of_dicts[1:]:
    df0 = pd.DataFrame.from_dict(d['properties'])
    dff = pd.concat([dff,df0])

In [42]:
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_colwidth', 100)

In [43]:
dff.head(1)

,Property ID,Title,Subtitle,Details,Description,Amenities,Agency,Agent,Titanium Agency?,Agency Video,Number of Images,Tour Video,Property Verified,Date Scraped,URL,Year of Construction,Servant Quarters,Parking Spaces,Double Glazed Windows,Central AC,Park,Jacuzzi
0,Flat 26526717,Beautiful Apartment Is Available For Sale In Reasonable Price,"Bahria Heights, Bahria Town Karachi, Karachi, Sindh","{'Purpose': 'For Sale', 'Type': 'Flat', 'Price': '52 Lakh', 'Location': 'Bahria Town Karachi, Ka...","Salaam Estate Offers:- Attractive Apartment Available for Sale at Bahria Heights, Bahria Town Fe...",Bedrooms : 2 Bathrooms : 2 Kitchens : 1 Lounge or Sitting Room Broadband Internet Access Satelli...,Salaam Estate & Builders,Rana Arsalan,True,True,10,False,False,"12-01-2020, 13:49:04",https://www.zameen.com/Property/bahria_town_karachi_bahria_heights_beautiful_apartment_is_availa...,NaN,NaN,NaN,No,No,No,No


In [44]:
dff.shape

(500208, 22)

### Remove duplicate listings

In [9]:
# dff = pd.read_csv('data/zameen-dec2020-clean.csv', dtype=np.str)

In [10]:
dff['Date Scraped'] = pd.to_datetime(dff['Date Scraped'])

In [11]:
dff['Date Scraped_date'] = dff['Date Scraped'].dt.date

In [70]:
dff['Date Scraped_date'].value_counts().reset_index().sort_values(by='index')[:10]

,index,Date Scraped_date
15,2020-12-01,127427
18,2020-12-02,96675
5,2020-12-03,149519
7,2020-12-04,146223
6,2020-12-05,148890
10,2020-12-06,141632
12,2020-12-07,139761
0,2020-12-08,153379
1,2020-12-09,153277
11,2020-12-10,139866


In [20]:
# remove dupes due to restarted scrapes
if len(dff[dff[['Property ID', 'Date Scraped_date']].duplicated()]) > 0:
    dff = dff[~dff[['Property ID', 'Date Scraped_date']].duplicated()].copy()

In [34]:
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', 500)

In [46]:
dupes = dff[dff[['Type', 'Locality', 'City', 'Area_marlas', 'Beds', 'Baths', 'Price_rupees_crores', 'Date Scraped', 'Agency']].duplicated(keep=False)].sort_values(by=['Date Scraped', 'City', 'Type', 'Locality', 'Area_marlas', 'Price_rupees_crores', 'Agency']).copy()



In [53]:
dupes.groupby('Type')['Property ID'].count().sort_values(ascending=False)

Type
Residential Plot     870316
House                444816
Flat                 252294
Upper Portion         93075
Lower Portion         41605
Commercial Plot       37663
Plot File             24721
Room                   1980
Farm House             1640
Penthouse              1279
Industrial Land         681
Agricultural Land       680
Plot Form                86
Plot                     52
Name: Property ID, dtype: int64

In [58]:
dupe_cols = ['Type', 'Locality', 'City', 'Area_marlas', 'Beds', 'Baths', 'Price_rupees_crores', 'Date Scraped', 'Agency']

In [59]:
len(dff[dff[['Type', 'Locality', 'City', 'Area_marlas', 'Beds', 'Baths', 'Price_rupees_crores', 'Date Scraped', 'Agency']].duplicated(keep='last')].sort_values(by=['Date Scraped', 'City', 'Type', 'Locality', 'Area_marlas', 'Price_rupees_crores', 'Agency']))



1191834

In [60]:
dff.shape

(4523277, 39)

In [61]:
1191834/4523277

0.2634890589278525

In [66]:
# drop 1,191,834 rows with duplicate listings
dff = dff[~dff[dupe_cols].duplicated()].copy()

In [67]:
dff.shape

(3331443, 39)

In [68]:
3331443+1191834

4523277

#### Expand details

In [56]:
dff['Details'].apply(lambda x: type(x))

0         <class 'dict'>
1         <class 'dict'>
2         <class 'dict'>
3         <class 'dict'>
4         <class 'dict'>
               ...      
140004    <class 'dict'>
140005    <class 'dict'>
140006    <class 'dict'>
140007    <class 'dict'>
140008    <class 'dict'>
Name: Details, Length: 500207, dtype: object

In [57]:
# dff.drop(columns=['Unnamed: 0'], inplace=True)
# dd = dff['Details'].apply(pd.Series)
# d = dff['Details'].map(eval)

In [58]:
new_cols = pd.json_normalize(dff['Details'])

In [60]:
new_cols.head()

,Purpose,Type,Price,Location,Area,Bedroom(s),Bath(s),Added
0,For Sale,Flat,52 Lakh,"Bahria Town Karachi, Karachi, Sindh",4.9 Marla,2,2,28 minutes ago
1,For Sale,House,86 Lakh,"Bahria Town Karachi, Karachi, Sindh",8 Marla,3,3,20 minutes ago
2,For Sale,Flat,50 Lakh,"Bahria Town Karachi, Karachi, Sindh",4.2 Marla,2,3,3 minutes ago
3,For Sale,House,3.8 Crore,"DHA Defence, Karachi, Sindh",4.8 Marla,4,5,16 minutes ago
4,For Sale,House,1.89 Crore,"Bahria Town Karachi, Karachi, Sindh",8 Marla,3,3,24 minutes ago


In [61]:
# need multiple cols, one for each key
new_cols.shape

(500207, 8)

In [62]:
# convert to arrays to speed up concatenation
dff_ = np.concatenate((dff.values, new_cols.values), axis=1)

In [63]:
col_names = dff.columns.append(new_cols.columns)

In [64]:
# merge with dff
dff = pd.DataFrame(dff_, columns=col_names)

In [65]:
# cols should increase by 8
dff.shape

(500207, 31)

In [66]:
dff.columns

Index(['Property ID', 'Title', 'Subtitle', 'Details', 'Description',
       'Amenities', 'Agency', 'Agent', 'Titanium Agency?', 'Agency Video',
       'Number of Images', 'Tour Video', 'Property Verified', 'Date Scraped',
       'URL', 'Year of Construction', 'Servant Quarters', 'Parking Spaces',
       'Double Glazed Windows', 'Central AC', 'Park', 'Jacuzzi',
       'Date Scraped_date', 'Purpose', 'Type', 'Price', 'Location', 'Area',
       'Bedroom(s)', 'Bath(s)', 'Added'],
      dtype='object')

#### Filter and rename columns

In [67]:
# drop Details, Description, Amenities
df = dff[['Property ID', 'Purpose', 'Type', 'Title', 'Subtitle', 'Location', 'Area', 'Price', 'Bedroom(s)', 'Bath(s)', 'Year of Construction', 'Servant Quarters', 'Parking Spaces', 'Double Glazed Windows', 'Central AC', 'Park', 'Jacuzzi', 'Agency', 'Agent', 'Titanium Agency?', 'Agency Video', 'Number of Images', 'Tour Video', 'Property Verified', 'Date Scraped', 'URL', 'Added']].copy()



In [68]:
df.rename(columns={'Bedroom(s)':'Beds', 'Bath(s)':'Baths'}, inplace=True)

In [69]:
pd.set_option('display.max_columns', 100)

In [70]:
df.head(2)

,Property ID,Purpose,Type,Title,Subtitle,Location,Area,Price,Beds,Baths,Year of Construction,Servant Quarters,Parking Spaces,Double Glazed Windows,Central AC,Park,Jacuzzi,Agency,Agent,Titanium Agency?,Agency Video,Number of Images,Tour Video,Property Verified,Date Scraped,URL,Added
0,Flat 26526717,For Sale,Flat,Beautiful Apartment Is Available For Sale In Reasonable Price,"Bahria Heights, Bahria Town Karachi, Karachi, Sindh","Bahria Town Karachi, Karachi, Sindh",4.9 Marla,52 Lakh,2,2,NaN,NaN,NaN,No,No,No,No,Salaam Estate & Builders,Rana Arsalan,True,True,10,False,False,2020-12-01 13:49:04,https://www.zameen.com/Property/bahria_town_karachi_bahria_heights_beautiful_apartment_is_availa...,28 minutes ago
1,House 26481937,For Sale,House,3 Beds Villa For Sale In Bahria Town Karachi,"Bahria Town - Precinct 31, Bahria Town Karachi, Karachi, Sindh","Bahria Town Karachi, Karachi, Sindh",8 Marla,86 Lakh,3,3,NaN,NaN,NaN,No,No,No,No,Sub Ki Property,SUB KI PROPERTY,True,False,11,False,False,2020-12-01 13:49:04,https://www.zameen.com/Property/bahria_town_karachi_bahria_town_precinct_31_3_beds_villa_for_sal...,20 minutes ago


In [71]:
df.shape

(500207, 27)

## Cleanup columns

In [72]:
df.isna().sum()

Property ID                   0
Purpose                       0
Type                          0
Title                         3
Subtitle                      0
Location                      0
Area                          0
Price                         0
Beds                          0
Baths                         0
Year of Construction     436354
Servant Quarters         445401
Parking Spaces           431285
Double Glazed Windows         0
Central AC                    0
Park                          0
Jacuzzi                       0
Agency                    55917
Agent                     55920
Titanium Agency?              0
Agency Video                  0
Number of Images              0
Tour Video                    0
Property Verified             0
Date Scraped                  0
URL                           0
Added                         0
dtype: int64

#### Year of Construction

In [73]:
len(df[df['Year of Construction'].isna()])/len(df)

0.8723468484047604

In [74]:
df['Year of Construction'] = df['Year of Construction'].astype('float')

In [75]:
df['Year of Construction'].describe()

count    63853.000000
mean      2011.951811
std        118.535923
min          0.000000
25%       2017.000000
50%       2019.000000
75%       2020.000000
max       7654.000000
Name: Year of Construction, dtype: float64

In [76]:
df['Year of Construction'] = np.where(df['Year of Construction'] == '0000', np.nan, df['Year of Construction'])

In [77]:
df['Year of Construction'] = np.where(df['Year of Construction'] == '5015', np.nan, df['Year of Construction'])

In [78]:
df['Year of Construction'] = np.where(df['Year of Construction'] == '2915', np.nan, df['Year of Construction'])

In [79]:
df['Year of Construction'] = np.where((df['Year of Construction'] > 2020) | (df['Year of Construction'] < 1900), np.nan, df['Year of Construction'])

In [80]:
df['Year of Construction'] = pd.to_datetime(df['Year of Construction'], format='%Y')

In [81]:
df['Year of Construction'].describe()

/Users/abdur_rehman/miniconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  """Entry point for launching an IPython kernel.


count                   61782
unique                     55
top       2020-01-01 00:00:00
freq                    28338
first     1909-01-01 00:00:00
last      2020-01-01 00:00:00
Name: Year of Construction, dtype: object

In [83]:
# very interesting!
# df[df['Year of Construction'] < '1990']

#### Date Scraped

In [84]:
df['Date Scraped'].value_counts()[:10]

2020-12-01 17:22:17    19
2020-12-30 20:43:31    19
2020-12-30 20:25:40    18
2020-12-30 20:12:51    18
2020-12-30 18:54:30    18
2020-12-15 08:08:42    18
2020-12-01 14:59:36    18
2020-12-15 06:55:14    17
2020-12-01 14:56:43    17
2020-12-15 07:08:17    17
Name: Date Scraped, dtype: int64

In [85]:
df['Date Scraped'] = pd.to_datetime(df['Date Scraped'])

In [86]:
df['Date Scraped'] = df['Date Scraped'].dt.date

#### Added Date

In [87]:
pd.set_option('min_rows', 100)

In [88]:
df['Added'].value_counts(normalize=True)[:20]

2 weeks ago     0.128195
1 week ago      0.115828
3 weeks ago     0.078262
1 month ago     0.073122
2 months ago    0.066714
3 days ago      0.055523
4 days ago      0.053286
2 days ago      0.049204
1 day ago       0.048640
4 weeks ago     0.048026
6 days ago      0.044370
5 days ago      0.041893
3 months ago    0.035317
4 months ago    0.018846
5 months ago    0.013776
4 hours ago     0.007487
6 months ago    0.006779
23 hours ago    0.006299
3 hours ago     0.006125
19 hours ago    0.006026
Name: Added, dtype: float64

#### Fix weeks in Added

In [89]:
df[df['Added'].str.contains('week|weeks')]['Added'].value_counts(normalize=True)

2 weeks ago    0.346182
1 week ago     0.312786
3 weeks ago    0.211340
4 weeks ago    0.129691
Name: Added, dtype: float64

In [90]:
df['Added'] = df['Added'].str.replace('ago', '')

In [91]:
df.reset_index(drop=True, inplace=True)

In [92]:
# check
# df[df.index.duplicated()]

,Property ID,Purpose,Type,Title,Subtitle,Location,Area,Price,Beds,Baths,Year of Construction,Servant Quarters,Parking Spaces,Double Glazed Windows,Central AC,Park,Jacuzzi,Agency,Agent,Titanium Agency?,Agency Video,Number of Images,Tour Video,Property Verified,Date Scraped,URL,Added


In [93]:
df['weeks'] = df[df['Added'].str.contains('week|weeks')]['Added'].apply(lambda x: x[0])

In [94]:
df['weeks'] = df[df['weeks'].notna()]['weeks'].astype(int)

In [96]:
df['weeks'].value_counts(normalize=True)

2.0    0.346182
1.0    0.312786
3.0    0.211340
4.0    0.129691
Name: weeks, dtype: float64

In [97]:
df['weeks'] = pd.to_timedelta(df['weeks'], unit='W')

#### Fix months in Added

In [98]:
df[df['Added'].str.contains('month|months')]['Added'].value_counts(normalize=True)

1 month       0.325806
2 months      0.297257
3 months      0.157363
4 months      0.083972
5 months      0.061383
6 months      0.030206
10 months     0.011981
9 months      0.010342
7 months      0.009549
8 months      0.006556
11 months     0.005585
Name: Added, dtype: float64

In [99]:
df['months'] = df[df['Added'].str.contains('month|months')]['Added'].apply(lambda x: x.split(' ')[0])

In [100]:
df['months'].value_counts(normalize=True)

1     0.325806
2     0.297257
3     0.157363
4     0.083972
5     0.061383
6     0.030206
10    0.011981
9     0.010342
7     0.009549
8     0.006556
11    0.005585
Name: months, dtype: float64

In [101]:
df['months'] = df[df['months'].notna()]['months'].astype(int)

In [102]:
df['months'] = df['months']*30

In [103]:
df['months'] = pd.to_timedelta(df['months'], unit='D')

#### Fix years in Added

In [104]:
df[df['Added'].str.contains('year|years')]['Added'].value_counts(normalize=True)

1 year      0.628099
2 years     0.236915
3 years     0.048209
7 years     0.038567
4 years     0.016529
6 years     0.013774
8 years     0.011019
5 years     0.006887
Name: Added, dtype: float64

In [105]:
df['years'] = df[df['Added'].str.contains('year|years')]['Added'].apply(lambda x: x.split(' ')[0])

In [106]:
df['years'].value_counts(normalize=True)

1    0.628099
2    0.236915
3    0.048209
7    0.038567
4    0.016529
6    0.013774
8    0.011019
5    0.006887
Name: years, dtype: float64

In [107]:
df['years'] = df[df['years'].notna()]['years'].astype(float)

In [108]:
df['years'] = df['years']*365.25

In [109]:
df['years'] = pd.to_timedelta(df['years'], unit='D')

#### Create timedelta

In [110]:
df['Added_td'] = np.timedelta64('NaT')

In [111]:
# add weeks and months to new col
df['Added_td'] = np.where(df['Added'].str.contains('week|weeks'), df['weeks'], df['Added_td'])
df['Added_td'] = np.where(df['Added'].str.contains('month|months'), df['months'], df['Added_td'])
df['Added_td'] = np.where(df['Added'].str.contains('year|years'), df['years'], df['Added_td'])

In [112]:
df['Added_td'].value_counts()

14 days 00:00:00      64124
7 days 00:00:00       57938
21 days 00:00:00      39147
30 days 00:00:00      36576
60 days 00:00:00      33371
28 days 00:00:00      24023
90 days 00:00:00      17666
120 days 00:00:00      9427
150 days 00:00:00      6891
180 days 00:00:00      3391
300 days 00:00:00      1345
270 days 00:00:00      1161
210 days 00:00:00      1072
240 days 00:00:00       736
330 days 00:00:00       627
365 days 06:00:00       456
730 days 12:00:00       172
1095 days 18:00:00       35
2556 days 18:00:00       28
1461 days 00:00:00       12
2191 days 12:00:00       10
2922 days 00:00:00        8
1826 days 06:00:00        5
Name: Added_td, dtype: int64

In [113]:
(df['Added_td'] == 'NaT').sum()

0

In [114]:
df['Added_td'] = df['Added_td'].astype(str)

In [115]:
df['Added_td'] = np.where(df['Added_td'] == 'NaT', df['Added'], df['Added_td'])

In [117]:
df['Added_td'][:25]

0          28 minutes 
1          20 minutes 
2           3 minutes 
3          16 minutes 
4          24 minutes 
5           3 minutes 
6           2 minutes 
7          50 minutes 
8          49 minutes 
9          43 minutes 
10         47 minutes 
11         19 minutes 
12         49 minutes 
13         24 minutes 
14    60 days 00:00:00
15          8 minutes 
16          8 minutes 
17          7 minutes 
18          8 minutes 
19         30 minutes 
20    60 days 00:00:00
21         29 minutes 
22         30 minutes 
23         29 minutes 
24         28 minutes 
Name: Added_td, dtype: object

In [118]:
df['Added_td'] = pd.to_timedelta(df['Added_td'])

#### Add Date Last Updated

In [119]:
df['Date Last Updated'] = np.where(df['Added_td'] < '1 day', df['Date Scraped'], np.timedelta64('NaT'))

In [120]:
df['Date Last Updated'] = np.where(df['Date Last Updated'].isnull(), df['Date Scraped'] - df['Added_td'], df['Date Last Updated'])



/Users/abdur_rehman/miniconda3/envs/ml/lib/python3.6/site-packages/pandas/core/arrays/datetimelike.py:1345: PerformanceWarning: Adding/subtracting object-dtype array to TimedeltaArray not vectorized
  PerformanceWarning,


In [121]:
df['Date Last Updated'].value_counts()[:5]

2020-12-01    40623
2020-11-24    35164
2020-11-17    30314
2020-11-30    22956
2020-12-15    22635
Name: Date Last Updated, dtype: int64

#### Add nans

In [122]:
# ASSUMPTION: missing values indicate 0
df['Servant Quarters'] = np.where(df['Servant Quarters'] == '', np.nan, df['Servant Quarters'])

In [123]:
df.isna().sum()

Property ID                   0
Purpose                       0
Type                          0
Title                         3
Subtitle                      0
Location                      0
Area                          0
Price                         0
Beds                          0
Baths                         0
Year of Construction     438425
Servant Quarters         445401
Parking Spaces           431285
Double Glazed Windows         0
Central AC                    0
Park                          0
Jacuzzi                       0
Agency                    55917
Agent                     55920
Titanium Agency?              0
Agency Video                  0
Number of Images              0
Tour Video                    0
Property Verified             0
Date Scraped                  0
URL                           0
Added                         0
weeks                    314975
months                   387944
years                    499481
Added_td                      0
Date Las

In [124]:
df.drop(columns=['Added_td', 'years', 'months', 'weeks'], inplace=True)

#### Title

In [126]:
# plots
df[df['Title'].isna()]

,Property ID,Purpose,Type,Title,Subtitle,Location,Area,Price,Beds,Baths,Year of Construction,Servant Quarters,Parking Spaces,Double Glazed Windows,Central AC,Park,Jacuzzi,Agency,Agent,Titanium Agency?,Agency Video,Number of Images,Tour Video,Property Verified,Date Scraped,URL,Added,Date Last Updated
30029,Residential Plot 23666223,For Sale,Residential Plot,None,"E-12/1, E-12, Islamabad, Islamabad Capital","E-12, Islamabad, Islamabad Capital",4 Marla,65 Lakh,-,-,NaT,NaN,NaN,No,No,No,No,Dubai Estates & Builder Pvt Ltd,Ch Waqas,False,False,0,False,False,2020-12-01,https://www.zameen.com/Property/e_12_e_12_1___-23666223-8177-2.html,5 months,2020-07-04
230453,Residential Plot 23666223,For Sale,Residential Plot,None,"E-12/1, E-12, Islamabad, Islamabad Capital","E-12, Islamabad, Islamabad Capital",4 Marla,65 Lakh,-,-,NaT,NaN,NaN,No,No,No,No,Dubai Estates & Builder Pvt Ltd,Ch Waqas,False,False,0,False,False,2020-12-15,https://www.zameen.com/Property/e_12_e_12_1___-23666223-8177-2.html,5 months,2020-07-18
393473,Residential Plot 23666223,For Sale,Residential Plot,None,"E-12/1, E-12, Islamabad, Islamabad Capital","E-12, Islamabad, Islamabad Capital",4 Marla,65 Lakh,-,-,NaT,NaN,NaN,No,No,No,No,Dubai Estates & Builder Pvt Ltd,Ch Waqas,False,False,0,False,False,2020-12-30,https://www.zameen.com/Property/e_12_e_12_1___-23666223-8177-2.html,6 months,2020-07-03


In [127]:
df = df[~df['Title'].isna()].copy()

#### Type

In [128]:
df['Type'].value_counts(normalize=True)

Residential Plot     0.439619
House                0.284422
Flat                 0.136718
Upper Portion        0.051603
Commercial Plot      0.034426
Lower Portion        0.029376
Plot File            0.014428
Farm House           0.002561
Industrial Land      0.001909
Room                 0.001873
Agricultural Land    0.001417
Penthouse            0.001144
Plot Form            0.000228
Plot                 0.000148
Home                 0.000128
Name: Type, dtype: float64

In [129]:
df['Type'] = np.where(df['Type'] == 'Home', 'House', df['Type'])

#### Add Locality, City

In [130]:
df['Locality'] = df['Location'].apply(lambda x: x.split(', ')[0])

In [131]:
df['City'] = df['Location'].apply(lambda x: x.split(', ')[-2])

#### Fix Area

In [132]:
df['Area_marlas'] = df['Area']

In [133]:
df['marlas'] = np.where(df['Area'].str.contains('Marla'), df['Area'], np.nan)
df['marlas'] = df['marlas'].str.replace(' Marla', '')
df['marlas'] = df['marlas'].astype(float)

In [134]:
df['marlas'].isna().sum()

146176

In [135]:
df['kanals'] = np.where(df['Area'].str.contains('Kanal'), df['Area'], np.nan)
df['kanals'] = df['kanals'].str.replace(' Kanal', '').str.replace(',', '')
df['kanals'] = df['kanals'].astype(float)
df['kanals'] = df['kanals'].fillna(0)
# 1 kanal = 20 marlas
df['kanals'] = df['kanals']*20

In [136]:
# check
len(df[df['kanals'] > 0])

146175

In [137]:
df['Area_marlas'] = np.where(df['marlas'].isna(), df['kanals'], df['marlas'])

In [138]:
df['Area_marlas'].describe()

count    5.002040e+05
mean     3.291099e+02
std      4.934991e+04
min      0.000000e+00
25%      5.000000e+00
50%      1.000000e+01
75%      2.000000e+01
max      9.544372e+06
Name: Area_marlas, dtype: float64

#### Price

In [139]:
df['Price_rupees'] = df['Price']

In [140]:
df['Price_rupees'] = df['Price_rupees'].str.replace(' Thousand', '000')

In [141]:
df['lakhs'] = np.where(df['Price'].str.contains('Lakh'), df['Price'], np.nan)
df['lakhs'] = df['lakhs'].str.replace(' Lakh', '')
df['lakhs'] = df['lakhs'].astype(float)
# 1 lakh = 100k
df['lakhs'] = df['lakhs']*100e3
df['lakhs'] = df['lakhs'].astype(str)

In [142]:
df['crores'] = np.where(df['Price'].str.contains('Crore'), df['Price'], np.nan)
df['crores'] = df['crores'].str.replace(' Crore', '')
df['crores'] = df['crores'].astype(float)
# 1 crore = 10 million
df['crores'] = df['crores']*10e6
df['crores'] = df['crores'].astype(str)

In [143]:
df['arabs'] = np.where(df['Price'].str.contains('Arab'), df['Price'], np.nan)
df['arabs'] = df['arabs'].str.replace(' Arab', '')
df['arabs'] = df['arabs'].astype(float)
# 1 arab = 1 billion
df['arabs'] = df['arabs']*1e9
df['arabs'] = df['arabs'].astype(str)

In [144]:
df['Price_rupees'] = np.where(df['Price_rupees'].str.contains('Lakh'), df['lakhs'], df['Price_rupees'])
df['Price_rupees'] = np.where(df['Price_rupees'].str.contains('Crore'), df['crores'], df['Price_rupees'])
df['Price_rupees'] = np.where(df['Price_rupees'].str.contains('Arab'), df['arabs'], df['Price_rupees'])

In [145]:
# plots
(df['Price_rupees'] == 'Contact for price').sum()

0

In [146]:
df[df['Price_rupees'] == 'Contact for price']

,Property ID,Purpose,Type,Title,Subtitle,Location,Area,Price,Beds,Baths,Year of Construction,Servant Quarters,Parking Spaces,Double Glazed Windows,Central AC,Park,Jacuzzi,Agency,Agent,Titanium Agency?,Agency Video,Number of Images,Tour Video,Property Verified,Date Scraped,URL,Added,Date Last Updated,Locality,City,Area_marlas,marlas,kanals,Price_rupees,lakhs,crores,arabs


In [147]:
# convert into nan, then drop them
df['Price_rupees'] = np.where(df['Price_rupees'] == 'Contact for price', np.nan, df['Price_rupees'])

In [148]:
df = df[~df['Price_rupees'].isna()].copy()

In [149]:
df['Price_rupees'] = df['Price_rupees'].astype(float).astype(int)

In [150]:
df['Price_rupees'].describe()

count    5.002040e+05
mean     2.563661e+07
std      2.455885e+08
min      1.000000e+00
25%      2.400000e+06
50%      8.000000e+06
75%      2.000000e+07
max      4.668000e+10
Name: Price_rupees, dtype: float64

In [71]:
df = dff.copy()

In [72]:
df.shape

(3331443, 39)

In [87]:
df['Price_rupees'] = df['Price_rupees'].astype(float)

In [88]:
df['Price_rupees_crores'] = df['Price_rupees']/1e7

In [89]:
df['Price_rupees_lakhs'] = df['Price_rupees']/1e5

In [90]:
df['Price_per_marla'] = df['Price_rupees_lakhs'] / df['Area_marlas'].astype(float)

In [91]:
df[['Price_rupees', 'Price_rupees_crores', 'Price_rupees_lakhs', 'Price_per_marla']].describe()

,Price_rupees,Price_rupees_crores,Price_rupees_lakhs,Price_per_marla
count,3.331337e+06,3.331337e+06,3.331337e+06,3.331337e+06
mean,2.853339e+07,2.853339e+00,2.853339e+02,1.646272e+01
std,2.469382e+08,2.469382e+01,2.469382e+03,5.730560e+01
min,1.000000e+00,1.000000e-07,1.000000e-05,2.619345e-11
25%,2.100000e+06,2.100000e-01,2.100000e+01,2.750000e+00
50%,8.682000e+06,8.682000e-01,8.682000e+01,1.050000e+01
75%,2.250000e+07,2.250000e+00,2.250000e+02,2.280000e+01
max,4.668000e+10,4.668000e+03,4.668000e+05,9.336000e+04


In [92]:
if len(df[df['Price_per_marla'] == np.inf]) > 0:
    df = df[df['Price_per_marla'] != np.inf].copy()

#### Year of Construction --> Age

In [151]:
df['Year of Construction'] = pd.to_datetime(df['Year of Construction'])

In [152]:
df['Year of Construction'].dtype

dtype('<M8[ns]')

In [153]:
# now – year of property construction
df['Age'] = np.datetime64('2021') -  df['Year of Construction']

In [154]:
df['Age'].dtype

dtype('<m8[ns]')

In [155]:
# convert into years
df['Age'] = df['Age'].apply(lambda x: x.days/365.25)

In [156]:
df['Age'] = df['Age'].round(2).astype(float)

In [157]:
df['Age'].value_counts(normalize=True)[:10]

1.0     0.458677
2.0     0.148975
3.0     0.097941
4.0     0.045758
6.0     0.044155
11.0    0.040157
5.0     0.034962
7.0     0.021867
9.0     0.017675
21.0    0.015684
16.0    0.013143
8.0     0.009954
13.0    0.009048
10.0    0.004985
Name: Age, dtype: float64

#### Add Freshness

In [158]:
df['Date Scraped'].value_counts(normalize=True)

2020-12-15    0.376442
2020-12-01    0.343656
2020-12-30    0.279902
Name: Date Scraped, dtype: float64

In [159]:
df['Date Scraped'] = df['Date Scraped'].astype('datetime64')
df['Date Last Updated'] = df['Date Last Updated'].astype('datetime64')

In [160]:
df['Freshness_days'] = df['Date Scraped'] - df['Date Last Updated']

In [161]:
df['Freshness_days'].value_counts(normalize=True)[:10]

14 days    0.128196
7 days     0.115829
0 days     0.110891
21 days    0.078262
30 days    0.073122
60 days    0.066715
3 days     0.055523
4 days     0.053286
2 days     0.049204
1 days     0.048640
Name: Freshness_days, dtype: float64

In [162]:
df['Freshness_days'] = df['Freshness_days'].apply(lambda x: x.days)

In [163]:
df['Freshness_days'].value_counts(normalize=True)[:10]

14    0.128196
7     0.115829
0     0.110891
21    0.078262
30    0.073122
60    0.066715
3     0.055523
4     0.053286
2     0.049204
1     0.048640
Name: Freshness_days, dtype: float64

In [164]:
df['Fresh'] = (df['Freshness_days'] == 0) * 1

In [165]:
fresh_counts = df.groupby('Property ID')['Fresh'].sum().reset_index().sort_values(by='Fresh', ascending=False)

In [166]:
fresh_counts[:5]

,Property ID,Fresh
83551,House 25871083,3
92319,House 26207070,3
210612,Residential Plot 26375653,3
88539,House 26081909,3
71200,House 24999717,3


In [167]:
# TODO: update according to data
# fix bad values (probably due to non-standardized scrape times)
# fresh_counts.iloc[0,1] = 8
# fresh_counts.iloc[1,1] = 8
# fresh_counts.iloc[2,1] = 8

In [168]:
df['Date Scraped'].nunique()

3

In [169]:
fresh_counts['Prob_Fresh'] = fresh_counts['Fresh']/df['Date Scraped'].nunique()

In [170]:
df = df.merge(fresh_counts[['Property ID', 'Prob_Fresh']], on='Property ID')

In [171]:
df['Prob_Fresh'].round(1).value_counts()

0.0    424103
0.3     71982
0.7      3828
1.0       291
Name: Prob_Fresh, dtype: int64

#### Fix Beds, Baths

In [172]:
df['Beds'] = np.where(df['Beds'] == '-', np.nan, df['Beds'])
df['Beds'] = df[df['Beds'].notna()]['Beds'].astype(float)

In [173]:
df['Baths'] = np.where(df['Baths'] == '-', np.nan, df['Baths'])
df['Baths'] = df[df['Baths'].notna()]['Baths'].astype(float)

In [175]:
df = df[['Property ID', 'Purpose', 'Type', 'Title', 'Subtitle', 'Location',
    'Area', 'Price', 'Beds', 'Baths', 'Year of Construction',
    'Servant Quarters', 'Parking Spaces', 'Double Glazed Windows',
    'Central AC', 'Park', 'Jacuzzi', 'Agency', 'Agent',
    'Titanium Agency?', 'Agency Video', 'Number of Images', 'Tour Video',
    'Property Verified', 'Date Scraped', 'URL', 'Added',
    'Date Last Updated', 'Locality', 'City', 'Area_marlas', 'Price_rupees', 
    'Age', 'Freshness_days', 'Prob_Fresh']].copy()